# Aurite API Agent: Functional Testing

This notebook is for functionally testing the `Aurite API Agent`. This agent uses an MCP server that wraps the framework's own API, allowing it to create and execute other components.

## 1. Setup
### Install the `aurite` Python package
First, ensure you have the `aurite-agents` package installed. If you are running this from the source repository, you can install it in editable mode with `pip install -e .` from the root directory.

In [71]:
%pip install -q -U "aurite"

Note: you may need to restart the kernel to use updated packages.


### Configure Your API Key
The `Aurite API Agent` needs an API key to communicate with the framework's API. For security, we'll load this from an environment variable.

In [72]:
import os
from getpass import getpass

try:
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from a .env file if it exists
except ImportError:
    print("dotenv not installed, skipping. Please ensure API_KEY is set manually.")

if "API_KEY" not in os.environ:
    os.environ["API_KEY"] = getpass("Enter your Aurite API key: ")

## 2. Initialize Aurite & Define Components

Next, we'll initialize the `Aurite` framework. Instead of loading from a file, we will define our `ClientConfig` and `AgentConfig` directly in Python.

In [73]:
from aurite import Aurite, AgentConfig, ClientConfig

aurite = Aurite()

# This command starts up the necessary background services.
await aurite.initialize()

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/testing/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/testing/aurite_config.json. Creating an empty project named 'aurite_config.json'.


INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


### Define the API MCP Server

Here, we define the configuration for our API-wrapping MCP server. We use an absolute path to the server script.

In [74]:
api_server_config = ClientConfig(
    name="api_mcp_server",
    server_path="/home/wilcoxr/workspace/aurite/aurite-agents/src/aurite/components/servers/orchestration/api_mcp_server.py",
    capabilities=["tools"],
    timeout=30.0
)

await aurite.register_client(api_server_config)
print("✅ API MCP Server registered successfully!")

INFO     [aurite.host.host] Attempting to dynamically register client: api_mcp_server
INFO     [aurite.host.host] Client 'api_mcp_server' dynamically registered and initialized successfully.


✅ API MCP Server registered successfully!


### Define the Aurite API Agent

Now, we create the agent that will use our new server. Its system prompt instructs it to use the tools provided.

In [75]:
from aurite import LLMConfig
creative_llm = LLMConfig(
    llm_id="gpt",
    provider="openai",
    model_name="gpt-4o",
    temperature=0.7,
)

api_agent_config = AgentConfig(
    name="Aurite Orchestration Agent",
    llm_config_id="gpt",
    mcp_servers=["api_mcp_server"],
    system_prompt="You are an AI Orchestration Agent built to interact with the Aurite AI Framework on behalf of users. Work with the available tools to manage components like agents and workflows. CRITICAL: You must use the exact tool names provided by the 'list_tools' function.")
await aurite.register_llm_config(creative_llm)
await aurite.register_agent(api_agent_config)
print("✅ Aurite Orchestration Agent registered successfully!")

✅ Aurite Orchestration Agent registered successfully!


### Helper for Displaying Output

In [76]:
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str):
  """Formats and displays the agent's response in a structured Markdown block."""

  output = f"""
  <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
    <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
      <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
        <span style=\"margin-right: 8px;\">🤖</span>
        Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
      </h3>
    </div>
    <div style=\"padding: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Your Query:</strong>
      </p>
      <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
        <em>\"{query}\"</em>
      </p>
      <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Result:</strong>
      </p>
      <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

## 3. Run the Test Case

Now, we'll run the agent with a two-step command: first create a new agent, then execute it.

In [ ]:
component_information_retrieval = "What are the available components? Can you list and explain them all?"

component_creation = "Create an agent that uses the national weather tool server to retrieve weather information. After you create the agent, ask it for the weather in New York."

multi_tool_agent_creation = "Help me create an agent that analyzes app store data, and then saves the data to a file as a plan. After you create the agent, ask it for the most popular app in the last month."
# Change this
user_message = multi_tool_agent_creation

# Run the agent with our query
agent_result = await aurite.run_agent(
    agent_name="Aurite Orchestration Agent",
    user_message=user_message
)

# Display the final response
display_agent_response(
    agent_name="Aurite Orchestration Agent",
    query=user_message,
    response=agent_result.primary_text
)

INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4o using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Aurite Orchestration Agent'...
INFO     [aurite.host.host] Executing tool 'list_registered_components' on client 'api_mcp_server' for agent 'Aurite Orchestration Agent'
